In [97]:
import boto3
import pandas as pd
import numpy as np
from IPython.display import display
import json

cluster_name="wahi_presto"
presto_min_cnt=15

emr = boto3.client('emr' , region_name='us-west-2')
clusters = emr.list_clusters()
presto_cluster = [i for i in clusters['Clusters'] if i['Name'] == cluster_name][0]


#emr_df = pd.DataFrame()
running_tasks_instances=emr.list_instances(ClusterId=presto_cluster['Id'],InstanceGroupTypes=['TASK'],InstanceStates=['RUNNING'])['Instances']
running_tasks_instances_df=pd.DataFrame(running_tasks_instances)
task_cnt=len(running_tasks_instances_df)

instance_groups=emr.list_instance_groups(ClusterId=presto_cluster['Id'])
instance_grp_id=[i['Id'] for i in instance_groups['InstanceGroups'] if i['InstanceGroupType'] =='TASK']
grp_id=' '.join(map(str, instance_grp_id))

print("Current count of task nodes:",task_cnt)

if task_cnt>presto_min_cnt:
    kill_cnt=task_cnt-presto_min_cnt
    print("Number of task nodes to be killed:",kill_cnt)
    kill_id=running_tasks_instances_df['Ec2InstanceId'].head(kill_cnt)
    
    emr.modify_instance_groups(ClusterId=presto_cluster['Id'],InstanceGroups=[{'InstanceGroupId':' '.join(map(str, instance_grp_id)),'EC2InstanceIdsToTerminate':kill_id.values.tolist() }])



Current count of task nodes: 20
Number of task nodes to be killed: 5
